## Html features. Visualization and experiments
Experimenting with the features extracted in the first notebook. 

### Motivation
Checking how the data is distributed, and also seeing whether we can predict some of the features from he others such as infering the type of the tag from the other features. Another experiment that could be run, might be trying to differentiating between different domains, and page types, using only the current features.

In [3]:
%matplotlib inline
# standard library
import itertools
import ast

from urllib.parse import urlparse

# pandas
import pandas as pd

# numpy, matplotlib, seaborn
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns


# skelearn
from sklearn.preprocessing import OneHotEncoder

# this styling is purely my preference
# less chartjunk
sns.set_context('notebook', font_scale=1.5, rc={'line.linewidth': 2.5})
sns.set(style='ticks', palette='Set2')

In [4]:
# load the data
df = pd.read_csv('../data/ecommerce-features.csv')
df.head()

,Unnamed: 0,depth,sibling_pos,tag,no_classes,has_id,no_children,has_text,classes,descendant1_no_nodes,...,ancestor2_depth,ancestor2_sibling_pos,ancestor2_tag,ancestor2_no_classes,ancestor2_has_id,ancestor2_no_children,ancestor2_has_text,ancestor2_classes,url,domain
0,0,1,0,html,0,False,2,False,[],2,...,0,0,NaN,0,False,0,False,[],https://marketplace-leads.emag.ro/?utm_source=...,marketplace-leads.emag.ro
1,1,2,0,head,0,False,96,False,[],96,...,0,0,NaN,0,False,0,False,[],https://marketplace-leads.emag.ro/?utm_source=...,marketplace-leads.emag.ro
2,2,3,0,meta,0,False,0,False,[],0,...,1,0,html,0,False,2,False,[],https://marketplace-leads.emag.ro/?utm_source=...,marketplace-leads.emag.ro
3,3,3,1,script,0,False,0,True,[],0,...,1,0,html,0,False,2,False,[],https://marketplace-leads.emag.ro/?utm_source=...,marketplace-leads.emag.ro
4,4,3,2,title,0,False,0,True,[],0,...,1,0,html,0,False,2,False,[],https://marketplace-leads.emag.ro/?utm_source=...,marketplace-leads.emag.ro


In [ ]:
def get_domain_from_url(url):
    """Returns the fully-qualified domain fo an url."""
    parsed_uri = urlparse(url)
    return parsed_uri.netloc

df['domain'] = df.url.apply(get_domain_from_url)

In [ ]:
# how many elements have text
df.has_text.value_counts().plot.bar()
plt.title('Has text')
sns.despine()

In [ ]:
# how many for each tag
plt.figure(figsize=(8,30))
df.groupby(['tag']).count()

# order them by the count(Descending)
df.tag.value_counts(ascending=True).plot.barh(rot=1, zorder=-3)
plt.grid(axis='x', color='white', linestyle='-', linewidth=1)
plt.xlabel('count')
plt.ylabel('tag')
sns.despine()

In [ ]:
df.no_classes.value_counts().plot.bar(rot=1, zorder=-3)
plt.grid(axis='y', color='white', linestyle='-', linewidth=1)
plt.xlabel('# of classes')
sns.despine()

In [ ]:
plt.figure(figsize=(8, 25))
df.groupby(['tag']).no_children.mean().sort_values().plot.barh(zorder=-3)
plt.grid(axis='x', color='white', linestyle='-', linewidth=1)
plt.xlabel('# of children')
sns.despine()

In [ ]:
# number of children distribution
df.no_children[df.no_children < 30].plot.bar(zorder=-3)
plt.xlabel('# of children')
plt.grid(axis='y', color='white', linestyle='-', linewidth=1)
sns.despine()

In [ ]:
df.depth.plot.hist(zorder=-3)
plt.xlabel('depth')
plt.grid(axis='y', color='white', linestyle='-', linewidth=1)
sns.despine()

In [ ]:
# count most frequent parent-child pairs of tags
pair_counts = df.groupby(['ancestor1_tag', 'tag']).size().sort_values(ascending=False)
pair_counts = pair_counts.reset_index()
pair_counts['asoc'] = pair_counts.ancestor1_tag + ' > '  + pair_counts.tag
pair_counts = pair_counts.rename(columns={0: 'count'})[['asoc', 'count']]
# pair_counts.index = pair_counts.asoc
pair_counts[75::-1].plot.barh(x='asoc', figsize=(10, 20), zorder=-3)
plt.grid(axis='x', color='white', linestyle='-', linewidth=1)
sns.despine()

In [ ]:
# count most frequent grandparent-parent-child tuples of tags
pair_counts = df.groupby(['ancestor2_tag', 'ancestor1_tag', 'tag']).size().sort_values(ascending=False)
pair_counts = pair_counts.reset_index()
pair_counts['asoc'] = ' > '.join([pair_counts.ancestor2_tag, pair_counts.ancestor1_tag, pair_counts.tag])
pair_counts = pair_counts.rename(columns={0: 'count'})[['asoc', 'count']]
# pair_counts.index = pair_counts.asoc
# only the first ones
pair_counts[75::-1].plot.barh(x='asoc', figsize=(10, 20), zorder=-3)
plt.grid(axis='x', color='white', linestyle='-', linewidth=1)
sns.despine()